In [1]:
import pandas as pd
import numpy as np

In [2]:
res15_valid = pd.read_csv('../probs_subs/check_res15_updated_file.txt')

res26_valid = pd.read_csv('../probs_subs/check_res26_updated_file.txt')

res8_valid = pd.read_csv('../probs_subs/check_res8_updated_file.txt')

In [3]:
res15_valid.sort_values(by='fname', inplace=True)
res26_valid.sort_values(by='fname', inplace=True)
res8_valid.sort_values(by='fname', inplace=True)

In [4]:
def softmax(x):
    scoreMatExp = np.exp(np.asarray(x))
    return scoreMatExp / scoreMatExp.sum(0)

In [5]:
prob_columns = [col for col in res15_valid.columns if 'prob' in col]

res15_valid[prob_columns] = res15_valid[prob_columns].apply(softmax)
res26_valid[prob_columns] = res26_valid[prob_columns].apply(softmax)
res8_valid[prob_columns] = res8_valid[prob_columns].apply(softmax)

In [6]:
res15_valid['cat'] = [s[-2] for s in res15_valid['fname'].str.rsplit('/')]
res15_valid['hash'] = [s[-1] for s in res15_valid['fname'].str.rsplit('/')]

res15_valid['fname'] = res15_valid['cat'] + '/' + res15_valid['hash']

In [7]:
res26_valid['cat'] = [s[-2] for s in res26_valid['fname'].str.rsplit('/')]
res26_valid['hash'] = [s[-1] for s in res26_valid['fname'].str.rsplit('/')]

res26_valid['fname'] = res26_valid['cat'] + '/' + res26_valid['hash']

res8_valid['cat'] = [s[-2] for s in res8_valid['fname'].str.rsplit('/')]
res8_valid['hash'] = [s[-1] for s in res8_valid['fname'].str.rsplit('/')]

res8_valid['fname'] = res8_valid['cat'] + '/' + res8_valid['hash']

In [8]:
res15_valid['cat'].value_counts()

up        272
left      267
off       262
right     259
yes       256
down      253
no        252
go        251
stop      249
on        246
eight      53
seven      40
five       40
one        39
six        38
zero       37
bed        37
three      36
four       34
sheila     33
house      33
nine       32
wow        30
two        30
happy      26
dog        23
cat        22
marvin     21
bird       20
tree       17
Name: cat, dtype: int64

In [9]:
(res26_valid['fname'] != res8_valid['fname'] ).sum()

0

In [10]:
(res15_valid['fname'] != res8_valid['fname'] ).sum()

0

In [11]:
res26_valid.drop(['fname', 'label'], axis=1, inplace=True)
res8_valid.drop(['fname', 'label'], axis=1, inplace=True)

In [12]:
res26_valid['max_prob'] = res26_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res8_valid['max_prob'] = res8_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res15_valid['max_prob'] = res15_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
label2word = {0: 'silence',
 1: 'unknown',
 2: 'yes',
 3: 'no',
 4: 'up',
 5: 'down',
 6: 'left',
 7: 'right',
 8: 'on',
 9: 'off',
 10: 'stop',
 11: 'go'}

res26_valid['max_prob_label'] = res26_valid['max_prob'].map(label2word)
res8_valid['max_prob_label'] = res8_valid['max_prob'].map(label2word)
res15_valid['max_prob_label'] = res15_valid['max_prob'].map(label2word)

In [14]:
res15_valid.columns = ['res15_' + col for col in res15_valid.columns]
res8_valid.columns = ['res8_' + col for col in res8_valid.columns]
res26_valid.columns = ['res26_' + col for col in res26_valid.columns]

In [15]:
common_valid = pd.concat([res8_valid, res26_valid, res15_valid], axis=1)

In [16]:
common_valid['true_label'] = common_valid['res15_label'].map(label2word)

In [17]:
voting = common_valid[[col for col in common_valid.columns if ('label' in col) and ('res15_label' not in col)]]

In [18]:
voting[(voting.res15_max_prob_label != voting.true_label) &
       ((voting.res15_max_prob_label == voting.true_label) |
       (voting.res15_max_prob_label == voting.true_label))]

,res8_max_prob_label,res26_max_prob_label,res15_max_prob_label,true_label


In [19]:
common_valid.columns

Index(['res8_prob_0', 'res8_prob_1', 'res8_prob_2', 'res8_prob_3',
       'res8_prob_4', 'res8_prob_5', 'res8_prob_6', 'res8_prob_7',
       'res8_prob_8', 'res8_prob_9', 'res8_prob_10', 'res8_prob_11',
       'res8_cat', 'res8_hash', 'res8_max_prob', 'res8_max_prob_label',
       'res26_prob_0', 'res26_prob_1', 'res26_prob_2', 'res26_prob_3',
       'res26_prob_4', 'res26_prob_5', 'res26_prob_6', 'res26_prob_7',
       'res26_prob_8', 'res26_prob_9', 'res26_prob_10', 'res26_prob_11',
       'res26_cat', 'res26_hash', 'res26_max_prob', 'res26_max_prob_label',
       'res15_fname', 'res15_prob_0', 'res15_prob_1', 'res15_prob_2',
       'res15_prob_3', 'res15_prob_4', 'res15_prob_5', 'res15_prob_6',
       'res15_prob_7', 'res15_prob_8', 'res15_prob_9', 'res15_prob_10',
       'res15_prob_11', 'res15_label', 'res15_cat', 'res15_hash',
       'res15_max_prob', 'res15_max_prob_label', 'true_label'],
      dtype='object')

In [20]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)] * \
                                         common_valid['res26_prob_' + str(prob)] * \
                                         common_valid['res15_prob_' + str(prob)])**(1/3)
    common_valid['gmean_' + str(prob)] = gmean

common_valid['gmean_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [21]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)]**2 * \
                                         common_valid['res26_prob_' + str(prob)]**2 * \
                                         common_valid['res15_prob_' + str(prob)]**2)**(1/3)
    common_valid['gmean_squared' + str(prob)] = gmean

common_valid['gmean_squared_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean_squared' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [22]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)]**2 * \
                                         common_valid['res26_prob_' + str(prob)]**2 * \
                                         common_valid['res15_prob_' + str(prob)])**(1/3)
    common_valid['gmean_squared_ex_res_fifthteen' + str(prob)] = gmean

common_valid['gmean_squared_ex_res_fifthteen_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean_squared_ex_res_fifthteen' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [25]:
for prob in range(12):
    mean = (common_valid['res8_prob_' + str(prob)] + \
                                         common_valid['res26_prob_' + str(prob)] + \
                                         common_valid['res15_prob_' + str(prob)])/3
    common_valid['usual_mean' + str(prob)] = mean

common_valid['mean_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'usual_mean' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [27]:
ensemble = common_valid[['res15_max_prob_label', 'gmean_max_prob_label', 
                         "gmean_squared_ex_res_fifthteen_max_prob_label",
                         'mean_max_prob_label', 'gmean_squared_max_prob_label', 
                         'true_label']]

In [28]:
ensemble[ensemble['res15_max_prob_label'] == ensemble['true_label']].shape

(2894, 6)

In [29]:
ensemble[ensemble['gmean_max_prob_label'] == ensemble['true_label']].shape

(1348, 6)

In [30]:
ensemble[ensemble['mean_max_prob_label'] == ensemble['true_label']].shape

(1213, 6)

In [31]:
ensemble[ensemble['gmean_squared_max_prob_label'] == ensemble['true_label']].shape

(1348, 6)

In [33]:
ensemble[ensemble['gmean_squared_ex_res_fifthteen_max_prob_label'] == ensemble['true_label']].shape

(769, 6)

# Вот тут ЖЁПА

In [37]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label)].shape

(313, 103)

In [38]:
common_valid[(common_valid.res8_max_prob_label == common_valid.true_label)].shape

(339, 103)

In [40]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)].shape

(24, 103)

In [43]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)][ensemble.columns]

,res15_max_prob_label,gmean_max_prob_label,gmean_squared_ex_res_fifthteen_max_prob_label,mean_max_prob_label,gmean_squared_max_prob_label,true_label
1888,go,unknown,unknown,unknown,unknown,unknown
1009,up,off,off,yes,off,yes
721,no,silence,silence,silence,silence,unknown
372,down,no,no,no,no,no
1974,silence,silence,silence,silence,silence,on
1497,yes,unknown,unknown,unknown,unknown,unknown
2232,stop,unknown,unknown,silence,unknown,unknown
3175,unknown,yes,yes,yes,yes,yes
2486,stop,no,no,no,no,up
2050,up,unknown,unknown,unknown,unknown,unknown


In [44]:
common_valid[(common_valid.res8_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)].shape

(26, 103)

In [51]:
common_valid[((common_valid.res8_max_prob_label == common_valid.true_label) |
             (common_valid.res15_max_prob_label == common_valid.true_label)) &
             (common_valid.res15_max_prob_label != common_valid.true_label)][ensemble.columns]

,res15_max_prob_label,gmean_max_prob_label,gmean_squared_ex_res_fifthteen_max_prob_label,mean_max_prob_label,gmean_squared_max_prob_label,true_label
782,stop,unknown,unknown,right,unknown,unknown
2673,unknown,left,left,right,left,right
2835,yes,no,no,no,no,unknown
1128,silence,left,left,unknown,left,unknown
1888,go,unknown,unknown,unknown,unknown,unknown
1442,off,up,up,up,up,up
3189,unknown,go,go,yes,go,go
2526,silence,off,off,off,off,unknown
3095,stop,unknown,unknown,unknown,unknown,unknown
1385,silence,unknown,unknown,down,unknown,down


In [53]:
common_valid.shape

(3208, 103)

In [55]:
# res15 дает 2894 из 3208
print(2894 / 3208)
print((2894 + 26) / 3208)

0.9021197007481296
0.9102244389027432


негусто, но что-то

In [41]:
res15 = pd.read_csv('../probs_subs/mix_model_res15.txt')

res26 = pd.read_csv('../probs_subs/mix_model_res26.txt')

res8 = pd.read_csv('../probs_subs/mix_model_res8.txt')

In [42]:
res15['label'] = res15[[col for col in res15.columns if 'prob' in col]].idxmax(axis=1).str.extract('(\d+)').astype(int)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [51]:
res15[['fname', 'label']].to_csv('res15.csv', index=False)

In [48]:
res15['label'] = res15['label'].map(label2word)

In [49]:
res15['label'] = res15['label'].map(label2word)

In [52]:
sol085 = pd.read_csv('../submissions/submission_mix_noises_updated.txt')

In [59]:
sol085.sort_values(by='fname', inplace=True)
sol085.head()

,fname,label
71424,clip_000044442.wav,no
158031,clip_0000adecb.wav,unknown
50617,clip_0000d4322.wav,unknown
86558,clip_0000fb6fe.wav,unknown
107514,clip_0001d1559.wav,unknown


In [60]:
res15.sort_values(by='fname', inplace=True)
res15.head()

,fname,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,prob_9,prob_10,prob_11,label
101340,clip_000044442.wav,-0.977384,-1.348111,2.131595,10.426859,-4.485994,5.713288,-2.758749,0.012697,-2.613747,-3.857974,-6.275019,3.524774,no
124379,clip_0000adecb.wav,-2.354270,7.805121,1.053384,-3.319956,-2.274884,1.480882,-3.501911,-0.289700,-1.475971,-3.287670,2.148622,2.826076,unknown
138910,clip_0000d4322.wav,-4.655570,7.682039,-3.166668,-1.656624,-5.331973,5.172206,-3.183746,-1.311582,-0.043383,-0.962119,3.987596,2.065382,unknown
109977,clip_0000fb6fe.wav,-0.059371,0.897257,0.027694,-0.824628,2.066972,-0.906270,0.790950,1.427714,-2.442889,-1.878847,0.811910,0.175236,up
134530,clip_0001d1559.wav,-3.948608,9.243481,-0.739165,-0.147731,-4.242608,1.361280,-3.195101,1.705395,1.064723,-2.168180,-2.613084,2.581058,unknown


In [61]:
res15['085_sol'] = sol085['label']

In [6]:
res15['sum'] = res15['prob_0'] + res15['prob_1'] + res15['prob_2'] + res15['prob_3'] + res15['prob_4'] + \
               res15['prob_5'] + res15['prob_6'] + res15['prob_7'] + res15['prob_8'] + res15['prob_9'] + \
               res15['prob_10'] + res15['prob_11']

In [9]:
res15['sum'].describe()

count    158538.000000
mean         -0.429559
std           0.802099
min          -4.248121
25%          -0.918870
50%          -0.423918
75%           0.041790
max           1.843294
Name: sum, dtype: float64

In [10]:
res26['sum'] = res26['prob_0'] + res26['prob_1'] + res26['prob_2'] + res26['prob_3'] + res26['prob_4'] + \
               res26['prob_5'] + res26['prob_6'] + res26['prob_7'] + res26['prob_8'] + res26['prob_9'] + \
               res26['prob_10'] + res26['prob_11']

In [11]:
res26['sum'].describe()

count    158538.000000
mean         -0.543636
std           0.612085
min          -3.676363
25%          -0.843397
50%          -0.468640
75%          -0.141315
max           1.217290
Name: sum, dtype: float64

In [14]:
res8['sum'] = res8['prob_0'] + res8['prob_1'] + res8['prob_2'] + res8['prob_3'] + res8['prob_4'] + \
               res8['prob_5'] + res8['prob_6'] + res8['prob_7'] + res8['prob_8'] + res8['prob_9'] + \
               res8['prob_10'] + res8['prob_11']

In [15]:
res8['sum'].describe()

count    158538.000000
mean          0.130686
std           0.394445
min          -1.761701
25%          -0.103157
50%           0.145098
75%           0.374886
max           1.561545
Name: sum, dtype: float64